In [2]:
import dill as pickle
from pluribus.game.evaluation import Evaluator
import math
import time
import random
import numpy as np

# maybe this implementation of EMD will be too slow..
from scipy.stats import wasserstein_distance
import matplotlib.pyplot as plt

In [3]:
with open('data/ehss_river.pkl', 'rb') as file:
    data = pickle.load(file)

In [4]:
# original deck the river centroids were created from
short_deck = data['short_deck']
# speedy look up for printing
eval_to_card = data['eval_to_card']
# river centroids
river_centroids = data['river_centroids']

# combos - I don't think I'll need all of them
on_flop = data['combos'][0]
on_turn = data['combos'][1]
on_river = data['combos'][2]

# expected hand strength on the river.. don't think I'll need it..
ehss_river = data['ehss_river'] 
evaluator = Evaluator()

In [5]:
def ncr(n,r):
    """
    helper function for calculating combination size
    n choose r"""
    return int(math.factorial(n)/(math.factorial(r)*math.factorial(n-r)))

In [6]:
print(f'expected turn: {ncr(10,2)*ncr(8,3)*ncr(5,1)}')

expected turn: 12600


In [7]:
print(f'my combos: {len(on_turn)}')

my combos: 12600


In [8]:
emd = [0]*len(on_river) # stats for expected hand strengths

In [9]:
# get data for expected hand rates on 4 card public board, not lossless

In [10]:
start = time.time()

# object for EMD on expected of hand strength (distribution)
ehs_distributions = []

for i, public in enumerate(on_turn): 
    available_cards = [x for x in short_deck if x not in public] # this is probably not a good idea
    random.shuffle(available_cards)
    
    # sample river cards and run a simulation
    ehs_distribution = np.zeros(len(river_centroids))
    for j in range(15):  
                        # probably want to increase this number?
                        # it's too small maybe for this toy problem

        river_card = random.sample(available_cards, 1)
        our_hand = list(public[:2])
        board = public[2:6]
        board = np.append(board, river_card).tolist()
        # if sample with river then error (because obvi)
        available_cards_river = [x for x in available_cards if x != river_card[0]] # oof
        
        our_hand_rank = evaluator.evaluate(
            board=board, 
            cards=our_hand,
        )
        
        # simulations will be run against randomly drawn opponent hands
        ehs=[0]*3
        for k in range(50): # will want to increase this?
            # sample from the available cards on the river
            opp_hand = random.sample(available_cards_river, 2)
            
            opp_hand_rank = evaluator.evaluate(
                    board=board,
                    cards=opp_hand,
            )
            
            # who wins?
            if our_hand_rank > opp_hand_rank: # maybe some mod magic here
                idx = 0
            elif our_hand_rank < opp_hand_rank:
                idx = 1
            elif our_hand_rank == opp_hand_rank:
                idx = 2
            
            # increment win rate for winner
            ehs[idx] += 1/50
        
        # get EMD for expected hand strength against each river centroid
        # to which does it belong?
        for idx, river_centroid in enumerate(river_centroids):
            emd = wasserstein_distance(ehs, river_centroid)
            
            if idx == 0:
                min_idx = idx
                min_emd = emd
            else:
                if emd < min_emd:
                    min_idx = idx
                    min_emd = emd
                    
        # ok, now increment the cluster to which it belongs - 
        ehs_distribution[min_idx] += 1/15 # could also probs be just a regular old integer
         
    ehs_distributions.append(ehs_distribution)
          
end = time.time()
print(end - start)

556.8644590377808


In [11]:
len(ehs_distributions)

12600

In [73]:
# simple kmeans algo - should I write from scratch?
from sklearn.cluster import KMeans

X = np.array(ehs_distributions)

km = KMeans(
    n_clusters=15, init='random',
    n_init=10, max_iter=300, 
    tol=1e-04, random_state=0
)
y_km = km.fit_predict(X)

In [74]:
# centers to be used to get data for EMD
centroids = km.cluster_centers_

In [75]:
data = {
    'ehs_distributions': ehs_distributions,
    'short_deck': short_deck,
    'combos': [on_flop, on_turn, on_river],
    'turn_centroids': centroids,
    'eval_to_card': eval_to_card
}

In [76]:
with open('data/ehs_dist_turn.pkl', 'wb') as file:
    pickle.dump(data, file)

In [77]:
# # to reopen
# with open('data/ehss_river.pkl', 'rb') as file:
#     data = pickle.load(file)

In [78]:
compare_hands = random.sample(list(on_turn[y_km==4]), 5)

In [79]:
# taking a look at them
# I think these are ones with a good chance of drawing, and maybe otherwise what?
for games in compare_hands:
    t = [eval_to_card[x] for x in games.tolist()[:2]]
    print("####Hand")
    print(t)
    t = [eval_to_card[x] for x in games.tolist()[2:]]
    print("####Board")
    print(t)

####Hand
[<Card card=[ace of diamonds ♦]>, <Card card=[7 of clubs ♣]>]
####Board
[<Card card=[ace of spades ♠]>, <Card card=[10 of clubs ♣]>, <Card card=[10 of diamonds ♦]>, <Card card=[4 of hearts ♥]>]
####Hand
[<Card card=[8 of hearts ♥]>, <Card card=[7 of clubs ♣]>]
####Board
[<Card card=[ace of spades ♠]>, <Card card=[ace of hearts ♥]>, <Card card=[10 of clubs ♣]>, <Card card=[ace of diamonds ♦]>]
####Hand
[<Card card=[8 of clubs ♣]>, <Card card=[10 of diamonds ♦]>]
####Board
[<Card card=[ace of spades ♠]>, <Card card=[8 of hearts ♥]>, <Card card=[7 of clubs ♣]>, <Card card=[3 of spades ♠]>]
####Hand
[<Card card=[8 of hearts ♥]>, <Card card=[10 of diamonds ♦]>]
####Board
[<Card card=[ace of diamonds ♦]>, <Card card=[10 of clubs ♣]>, <Card card=[7 of clubs ♣]>, <Card card=[4 of hearts ♥]>]
####Hand
[<Card card=[ace of spades ♠]>, <Card card=[8 of clubs ♣]>]
####Board
[<Card card=[8 of hearts ♥]>, <Card card=[3 of spades ♠]>, <Card card=[10 of clubs ♣]>, <Card card=[4 of hearts ♥]>]


In [80]:
# ok, this looks understandable
print(np.array(ehs_distributions)[y_km==4][:10])

[[0.13333333 0.06666667 0.         0.13333333 0.         0.
  0.         0.         0.06666667 0.         0.2        0.06666667
  0.06666667 0.13333333 0.13333333]
 [0.06666667 0.13333333 0.         0.33333333 0.         0.13333333
  0.         0.         0.13333333 0.         0.06666667 0.
  0.06666667 0.         0.06666667]
 [0.         0.         0.         0.33333333 0.         0.06666667
  0.         0.         0.13333333 0.         0.2        0.06666667
  0.06666667 0.06666667 0.06666667]
 [0.06666667 0.13333333 0.         0.33333333 0.         0.13333333
  0.         0.         0.2        0.         0.06666667 0.
  0.         0.         0.06666667]
 [0.06666667 0.06666667 0.         0.26666667 0.         0.13333333
  0.         0.         0.         0.         0.2        0.
  0.06666667 0.         0.2       ]
 [0.13333333 0.13333333 0.         0.2        0.         0.06666667
  0.         0.         0.2        0.         0.13333333 0.06666667
  0.06666667 0.         0.        ]


In [81]:
# ok, this looks understandable
print(np.array(ehs_distributions)[y_km==0][:10])

[[0.06666667 0.06666667 0.         0.46666667 0.         0.
  0.         0.         0.06666667 0.         0.         0.06666667
  0.         0.2        0.06666667]
 [0.         0.         0.         0.66666667 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.33333333 0.        ]
 [0.         0.         0.         0.66666667 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.33333333 0.        ]
 [0.         0.         0.         0.46666667 0.         0.
  0.         0.         0.         0.         0.         0.06666667
  0.         0.46666667 0.        ]
 [0.         0.         0.         0.53333333 0.         0.06666667
  0.         0.         0.         0.         0.         0.06666667
  0.         0.33333333 0.        ]
 [0.13333333 0.         0.         0.46666667 0.         0.
  0.         0.         0.06666667 0.         0.         0.
  0.         0.26666667 0.06666667]
 [0.         0.         0.      

In [2]:
import dill as pickle
with open('data/information_abstraction.pkl', 'rb') as file:
    data2 = pickle.load(file)

In [3]:
data2

{'_cards': [<Card card=[3 of hearts ♥]>,
  <Card card=[7 of diamonds ♦]>,
  <Card card=[3 of spades ♠]>,
  <Card card=[jack of clubs ♣]>,
  <Card card=[9 of hearts ♥]>,
  <Card card=[9 of spades ♠]>,
  <Card card=[6 of spades ♠]>,
  <Card card=[king of hearts ♥]>,
  <Card card=[5 of hearts ♥]>,
  <Card card=[8 of hearts ♥]>],
 '_evals': [139523,
  2114829,
  135427,
  33589533,
  8398611,
  8394515,
  1053707,
  134228773,
  533255,
  4204049],
 '_evals_to_cards': {139523: <Card card=[3 of hearts ♥]>,
  2114829: <Card card=[7 of diamonds ♦]>,
  135427: <Card card=[3 of spades ♠]>,
  33589533: <Card card=[jack of clubs ♣]>,
  8398611: <Card card=[9 of hearts ♥]>,
  8394515: <Card card=[9 of spades ♠]>,
  1053707: <Card card=[6 of spades ♠]>,
  134228773: <Card card=[king of hearts ♥]>,
  533255: <Card card=[5 of hearts ♥]>,
  4204049: <Card card=[8 of hearts ♥]>},
 'starting_hands': array([[   139523,   2114829],
        [   139523,    135427],
        [   139523,  33589533],
        [ 

In [16]:
import random
compare_hands = random.sample(list(data2['_flop_potential_aware_distributions'][data2['_flop_clusters']==0]), 5)
# taking a look at them
for games in compare_hands:
    t = [data2['_evals_to_cards'][x] for x in games.tolist()[:2]]
    print("####Hand")
    print(t)
    t = [data2['_evals_to_cards'][x] for x in games.tolist()[2:]]
    print("####Board")
    print(t)

KeyError: 0.2

In [17]:
import math

In [18]:
def ncr(n,r):
    """
    helper function for calculating combination size
    n choose r"""
    return int(math.factorial(n)/(math.factorial(r)*math.factorial(n-r)))

In [19]:
ncr(52,3)*ncr(50,1)*ncr(49,1)

54145000

In [20]:
1/54145000

1.8468926031951242e-08